In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as Dataset
import torchvision.transforms as Transforms

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Dataset

In [29]:
#Download the dataset
train_dataset = Dataset.EMNIST(root="./data", train=True, transform=Transforms.ToTensor(), split="balanced", download=True)
test_dataset = Dataset.EMNIST(root="./data", train=False, transform=Transforms.ToTensor(), split="balanced", download=True)

 48%|████▊     | 266960896/561753746 [26:13<28:57, 169654.12it/s]  


KeyboardInterrupt: 

In [16]:
VGG16 = [64, 64, 'M', 128,128,'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

In [18]:
NUM_EPOCH = 10
NUM_CLASSES = 1000

In [19]:
class VGGNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000, architecture=VGG16):
        super(VGGNet, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layer(VGG16)
        self.fcs = nn.Sequential(
            nn.Linear(512*7*7, 4096), # 7=224/(2**5), here 5 cause 5 Maxpool Layers 
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,num_classes)
        )
        self.softMax = nn.Softmax(dim=1) # not sure


    def forward(self, x):
        out = self.conv_layers(x)
        out = out.reshape(out.shape[0], -1)
        out = self.fcs(out)
        out = self.softMax(out)
        return out

    def create_conv_layer(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3,3),stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]

        return nn.Sequential(*layers)
        


In [20]:
model = VGGNet(architecture=VGG16).to(device)
# model

In [21]:
def test():
    x = torch.randn(10, 3, 224, 224)
    print(x.shape)

    result = model(x.to(device))
    # print(result.sum())
    return result

fp = test()


torch.Size([10, 3, 224, 224])


In [22]:
fp.sum()

tensor(10., grad_fn=<SumBackward0>)